In [ ]:
import pandas as pd
import supy as sp
import matplotlib.pyplot as plt


# # from Utilities.db_functions import (read_DB, decide_country_or_region, fill_SUEWS_NonVeg, fill_SUEWS_NonVeg_typologies,  fill_SUEWS_Water, 
#                                      fill_SUEWS_Veg, fill_SUEWS_S now, fill_SUEWS_AnthropogenicEmission, 
#                                      fill_SUEWS_profiles, blend_SUEWS_NonVeg, save_SUEWS_txt, save_snow, 
#                                      save_NonVeg_types, save_SiteSelect, presave, read_morph_txt, surf_df_dict)
db_path = r'C:\GitHub\suews_prepare_database\Input\database.xlsx'


In [27]:
pp = r'C:\Users\xbacos\OneDrive - University of Gothenburg\Artikel_4\OUT\OSept'

nonveg_p = pp + '/SUEWS_NonVeg.txt'

df_nonveg = pd.read_csv(nonveg_p, skiprows =-2, delimiter='\t',)


veg_p = pp + '/SUEWS_Veg.txt'

df_veg = pd.read_csv(veg_p, skiprows =-2, delimiter='\t')

In [ ]:
len(df_veg.iloc[1,0])

In [ ]:
len(df_nonveg.iloc[1,0])

In [ ]:
df_nonveg

In [ ]:
# Original
db_dict= read_DB(db_path)


In [2]:
in_folder = r'C:\Users\xbacos\OneDrive - University of Gothenburg\Artikel_4\OUT'

# in_folder = r'C:\GitHub\SUEWS_DB_Typology_test\data\sample_data'
rc = in_folder + '/RunControl.nml'  

In [ ]:
sp.show_version()

In [ ]:
import pandas as pd
import supy as sp

in_folder = r'C:\Users\xbacos\OneDrive - University of Gothenburg\Artikel_4\OUT'

# in_folder = r'C:\GitHub\SUEWS_DB_Typology_test\data\sample_data'
rc = in_folder + '/RunControl.nml'  

df_state_init = sp.init_supy(rc)